# Ch.12 Generative deep learning

**Chapter Contents:**
* Text generation
* Neural style transfer
* Variational autoencoders
* Generative adversarial networks


## Text generation

RNN을 사용하여 시퀀스 데이터를 생성하는 방법을 살펴봅니다.

예를 들어 텍스트 생성

정확히 동일한 기술을 모든 종류의 시퀀스 데이터로 일반화할 수 있습니다.

* 음표(musical notes )(음악)
* 브러시 스트로크 데이터의 시계열(그림)
* ...


### A brief history of generative deep learning for sequence generation

### How do you generate sequence data?

이전 토큰을 입력으로 사용하여 다음 토큰을 예측하도록 모델(Transformer 또는 RNN) 훈련

<img src="https://drek4537l1klr.cloudfront.net/chollet2/Figures/12-01.png" width="400"><p style="text-align:center">Figure 12.1 The process of word-by-word text generation using a language model.</p>


### The importance of the sampling strategy

순진한 접근 방식: **greedy sampling** (가장 가능성이 높은 다음 토큰 선택)

반복적이고 **예측 가능한** 문자열(일관된 언어처럼 보이지 않음)

더 나은 접근 방식: **무작위성(randomness)** 도입(stochastic sampling)

예: 토큰의 확률 0.3: 선택할 확률 30%.

**Softmax** : 확률 분포를 위한 좋은 후보입니다. 그러나 임의성의 정도를 제어할 수 없음


솔루션: **softmax temperature**:

> 주어진 temperature value,
>
> 새로운 확률 분포는 원래 분포에서 계산됩니다(모델의 softmax 출력).
>
> 다음과 같은 방법으로 가중치를 재조정합니다.

**Reweighting a probability distribution to a different temperature**

In [1]:
import numpy as np
def reweight_distribution(original_distribution, temperature=0.5):
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    return distribution / np.sum(distribution)

<img src="https://drek4537l1klr.cloudfront.net/chollet2/Figures/12-02.png" width="400"><p style="text-align:center">Figure 12.2 Different reweightings of one probability distribution. Low temperature = more deterministic, high temperature = more random.</p>

### Implementing text generation with Keras

#### Preparing the data

**Downloading and uncompressing the IMDB movie reviews dataset**

In [0]:
# !wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
# !tar -xf aclImdb_v1.tar.gz

**Creating a dataset from text files (one file = one sample)**

In [2]:
import tensorflow as tf
from tensorflow import keras
dataset = keras.utils.text_dataset_from_directory(
    directory="aclImdb", label_mode=None, batch_size=256)
dataset = dataset.map(lambda x: tf.strings.regex_replace(x, "<br />", " "))

Found 55006 files belonging to 1 classes.


**Preparing a `TextVectorization` layer**

In [3]:
from tensorflow.keras.layers import TextVectorization

sequence_length = 100
vocab_size = 15000
text_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
text_vectorization.adapt(dataset)

**Setting up a language modeling dataset**

In [4]:
def prepare_lm_dataset(text_batch):
    vectorized_sequences = text_vectorization(text_batch)
    x = vectorized_sequences[:, :-1]
    y = vectorized_sequences[:, 1:]
    return x, y

lm_dataset = dataset.map(prepare_lm_dataset, num_parallel_calls=4)

#### A Transformer-based sequence-to-sequence model

일정 수의 초기 단어가 주어졌을 때 다음 단어에 대한 확률 분포를 예측하도록 모델을 훈련시킵니다.

추론(Inference):

* 프롬프트로 모델에 피드.
* 다음 단어 샘플
* 해당 단어를 프롬프트에 다시 추가
* 짧은 단락이 생성될 때까지 반복

우리는 지난 장의 sequence-to-sequence 아키텍처를 사용할 것입니다.


<img src="https://drek4537l1klr.cloudfront.net/chollet2/Figures/12-03.png" width="340"><p style="text-align:center">Figure 12.3 Compared to plain next-word prediction, sequence-to-sequence modeling simultaneously optimizes for multiple prediction problems.</p>

In [6]:
import tensorflow as tf
from tensorflow.keras import layers

class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
          num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
          num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super(TransformerDecoder, self).get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

**A simple Transformer-based language model**

In [8]:
from tensorflow.keras import layers
embed_dim = 256
latent_dim = 2048
num_heads = 2

inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, x)
outputs = layers.Dense(vocab_size, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop")

### A text-generation callback with variable-temperature sampling

**The text-generation callback**

In [9]:
import numpy as np

tokens_index = dict(enumerate(text_vectorization.get_vocabulary()))

def sample_next(predictions, temperature=1.0):
    predictions = np.asarray(predictions).astype("float64")
    predictions = np.log(predictions) / temperature
    exp_preds = np.exp(predictions)
    predictions = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, predictions, 1)
    return np.argmax(probas)

class TextGenerator(keras.callbacks.Callback):
    def __init__(self,
                 prompt,
                 generate_length,
                 model_input_length,
                 temperatures=(1.,),
                 print_freq=1):
        self.prompt = prompt
        self.generate_length = generate_length
        self.model_input_length = model_input_length
        self.temperatures = temperatures
        self.print_freq = print_freq

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.print_freq != 0:
            return
        for temperature in self.temperatures:
            print("== Generating with temperature", temperature)
            sentence = self.prompt
            for i in range(self.generate_length):
                tokenized_sentence = text_vectorization([sentence])
                predictions = self.model(tokenized_sentence)
                next_token = sample_next(predictions[0, i, :])
                sampled_token = tokens_index[next_token]
                sentence += " " + sampled_token
            print(sentence)

prompt = "This movie"
text_gen_callback = TextGenerator(
    prompt,
    generate_length=50,
    model_input_length=sequence_length,
    temperatures=(0.2, 0.5, 0.7, 1., 1.5))

**Fitting the language model**

In [13]:
model.fit(lm_dataset, epochs=50, callbacks=[text_gen_callback])

Epoch 1/50
215/215 [==============================] - ETA: 0s - loss: 3.9744== Generating with temperature 0.2
This movie was had a a pleasure very to young see italian the cinema young with man a i nervous always and cried she my was heart so for damned both he male thrilled lead me is to wise cry and to a be young a woman woman active in past the
== Generating with temperature 0.5
This movie is has made the my 3rd jones 4th scifi man movie series in than every [UNK] seasons the of story personal or story so but farley the had memory got of the his failure [UNK] fails talent [UNK] the john talking richly of deserved episode time with i the can
== Generating with temperature 0.7
This movie is is about the rape true spike thing jones has may to be be one blamed of when [UNK] sassy robot one jones tries one his from hardest [UNK] to that catch went parts to 2 england million to bc live the ride perfect to soil its there life with
== Generating with temperature 1.0
This movie movie was is

Epoch 6/50
215/215 [==============================] - ETA: 0s - loss: 3.7829== Generating with temperature 0.2
This movie movie is is real definitely from in the the losers hallmark it movie has adopted little at girls all also into look the at movie the put movie together together and this at movie the it fact does that move there it is makes best everything funny the was movie
== Generating with temperature 0.5
This movie is has dumb one enough scene to which describe is it funny is from not the to first be scene jackie in chan the comedy film and is the good same and as the his sweet other as [UNK] the but evil most spirits of are the truly south funny
== Generating with temperature 0.7
This movie film may is have not considered [UNK] as to [UNK] be but forgotten it often is is seen one in film the with ones good and acting the is film simply [UNK] beautiful [UNK] cinematography is is all extremely so amateur powerful though the and performances mesmerizing one settings of
== Genera

Epoch 11/50
215/215 [==============================] - ETA: 0s - loss: 3.6401== Generating with temperature 0.2
This movie has was everything not getting only on if name it were was considered possible bad to to save be this artistic bad director this tim has burton been hills distant out past a about tax 4 movie years movies ago are and at they dead have with fading more sets
== Generating with temperature 0.5
This movie was sounded done better far than as most well movies as i kids have from seen major it production should values watch be and around enjoy of the the culture second and half its are ending really that good good in the places category where then clint they eastwood might
== Generating with temperature 0.7
This movie movie just should isnt have welcome read at andor least cheaper this costs movie oh is my a objective neat account job be thats carried adding on so [UNK] many isnt of going course to there save like the this catching however film i fails think because it of

Epoch 16/50
215/215 [==============================] - ETA: 0s - loss: 3.5330== Generating with temperature 0.2
This movie is still all a about waste life of in two your hours life watching the it writers felt not like even the funny single screenplay guy dialogues and make his it whole almost idea completely of  the famous [UNK] pointless [UNK] with [UNK] metaphysical conditions the hell are two
== Generating with temperature 0.5
This movie time s out my of friend all despite the its [UNK] popularity of of the the time american i release also to a the copy early of original the or movie other in a [UNK] word cast it is has voiced faded by now and and im [UNK] [UNK] the
== Generating with temperature 0.7
This movie is starts absolutely out hilarious pretty this good must but also it the plays other for members the especially great in parts this paul movie played it nice was to entertaining see and it from was paul watch [UNK] but i at have the an same laughs person  from
== Generating 

Epoch 21/50
215/215 [==============================] - ETA: 0s - loss: 3.4479== Generating with temperature 0.2
This movie is grabs really which and can drags only strange begins if for you a have suspense the then slow lets it just is see fun a other quality couple movie of cuts times and and action the too dragged doesnt there stop arent until many the scenes characters until enters
== Generating with temperature 0.5
This movie sequel is starts exactly badly great into plus a the bit title too means haha but the there fun are in some the things sudden looks most like older a than normal it husband does pays not attention mendes to for [UNK] the decides rights to there use to a
== Generating with temperature 0.7
This movie was captures terrible before at christmas the time time and of 2 time souls italians it i fade must into have the been money plain for crap land even fine though actors the are hole much and like their are comparatively all limited the illegal actors take seems thei

...

This movie is was very described funny in by a everyone fear else is hates not it just looked about like every a other movie [UNK] of [UNK] it character even is comes totally as inappropriate funny for although children although can i actually love think it too really is is a
215/215 [==============================] - 63s 291ms/step - loss: 3.3327
Epoch 31/50
215/215 [==============================] - ETA: 0s - loss: 3.3223== Generating with temperature 0.2
This movie soso is [UNK] a than [UNK] stick but to some the nostalgia action comedy scenes elements and into slapstick the [UNK] cliches battles whenever are were down about the near plot watchers [UNK] did of look you worthy know of we the saw only low because budget the was [UNK]
== Generating with temperature 0.5
This movie movie was was obviously [UNK] [UNK] [UNK] no story budget didnt testing know buildings the to [UNK] ups are multiple bad personalities people ive stereotyped never characters chorus [UNK] interesting lots each

This movie is home about movie an no older incident [UNK] who clear appears her to two be young [UNK] wife who who is continues out to of follow the her footsteps kindly fall themed in bathing china suit and the [UNK] story sized paid tough to doesnt nods mean to mutants
== Generating with temperature 1.5
This movie movie was reminds [UNK] me vancouver when of kids the watch what the they experience agreed thought to it try was and going upsetting to towards make the for viewing a the tv trailer show and and its have [UNK] ta some or [UNK] her [UNK] survival it schedule was
215/215 [==============================] - 62s 286ms/step - loss: 3.2837
Epoch 36/50
215/215 [==============================] - ETA: 0s - loss: 3.2749== Generating with temperature 0.2
This movie is is just about because perfect it performances is are the cool only the mom cast was members great of especially freely [UNK] the and theme [UNK] song and looks [UNK] great is but excellent the in movie this the is film z

This movie is is outstanding not and a the movie dream for sequence those and who hope went so by far though the not story too his the anywhere only as thing we i have have was seen one going movie to recently see the the first acting 50 was times british
== Generating with temperature 1.0
This movie film had certainly many setpieces competent that counts stunning was [UNK] a to [UNK] be thriller independent and professional the entertainment sparkling value results is in only a after good just for an the intriguing cameo atmosphere by of [UNK] the by [UNK] [UNK] evans kay recent [UNK] hits and
== Generating with temperature 1.5
This movie episode tells just a about really that [UNK] the son movie his is father somewhat who a has comedy lived and many it people also realize receives that a life few when things it are ends total completely of incorrect course the there two is kids laugh who when get
215/215 [==============================] - 62s 286ms/step - loss: 3.2419
Epoch 41/50
215

This movie movie is now bad boring because [UNK] i cheesy cannot some imagine funny zombies bits do at not widowed because and people goes stay around afraid there when will they crack grow at up you the should songs be are very worth unpleasant their people time are cannon on fodder
== Generating with temperature 0.7
This movie film was deals just a three mystery year robbers old colonel hostages who ask wanted his their claude jobs terrorists by brooding taking william drugs [UNK] into [UNK] closing role people were shooting forced up into mentally detective phones [UNK] in in dodge the the murky war duty of hanging
== Generating with temperature 1.0
This movie is took one me as by a my southern brother 2003 etc man boy and take the something tagline new what boy the did film i uwe feel boll that wood he had was [UNK] in of scream the box woods of at the the time time that the id
== Generating with temperature 1.5
This movie series really is doesnt exactly have resemble to many say [

This movie film was is a a fantastic welcome film fun with [UNK] lots [UNK] of and violence lots in of it sexual is sexy actually and if very there accurate is writing nothing it serious is about very being sexy [UNK] and it [UNK] must to i do have not any
== Generating with temperature 0.7
This movie is says getting as your the child movie and is watch a while year u and will will get definitely a a better [UNK] after [UNK] a of while the there movie is is worthy a of stop being watching there to will work be soon spoilers to ahead what
== Generating with temperature 1.0
This movie film details is in a the channel documentary shorts reminded are me seen a by lot some of of the the clips characters of are the excellent one of the the most films unrealistic that parts doesnt are come well long [UNK] and does does very a well good as
== Generating with temperature 1.5
This movie review would was be much a better fat movie [UNK] if i you had were known author for beach the [UNK] book lose

== Generating with temperature 0.2

This movie had is what a it stupid had people no are idea from unrelated a to book that that would i make just a watch film her with at a home young run woman apart hopkins at [UNK] [UNK] other looks than like an a actual [UNK] portrayal the by camera

== Generating with temperature 0.5

This movie film was is a a fantastic welcome film fun with [UNK] lots [UNK] of and violence lots in of it sexual is sexy actually and if very there accurate is writing nothing it serious is about very being sexy [UNK] and it [UNK] must to i do have not any

== Generating with temperature 0.7

This movie is says getting as your the child movie and is watch a while year u and will will get definitely a a better [UNK] after [UNK] a of while the there movie is is worthy a of stop being watching there to will work be soon spoilers to ahead what

== Generating with temperature 1.0

This movie film details is in a the channel documentary shorts reminded are me seen a by lot some of of the the clips characters of are the excellent one of the the most films unrealistic that parts doesnt are come well long [UNK] and does does very a well good as

== Generating with temperature 1.5

This movie review would was be much a better fat movie [UNK] if i you had were known author for beach the [UNK] book loses and his sight family it and was his very grave good that ernest bad [UNK] guy made and a the great film robert of [UNK] [UNK] the



Low temperature: 지루하고 반복적인 텍스트(때로는 루프에 갇힘)

Higher temperatures: 생성된 텍스트가 더 흥미롭고 놀랍고 창의적이 됩니다.

Very high temperature: 국부 구조가 무너지기 시작하고 출력이 대체로 무작위로 보입니다.

### Wrapping up